In [1]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Violations = pd.read_csv("/Users/Carmijh0/Desktop/CSV's/Property_Standards_Violations.csv")
Standards = pd.read_csv("/Users/Carmijh0/Desktop/CSV's/Property_Standards_Violations.csv")

In [2]:
Violations

,Request #,Date Received,Property APN,Property Address,City,State,ZIP,Property Owner,Complaint Source,Reported Problem,Status,Council District,Last Activity Date,Last Activity,Last Activity Result,Violations Noted,Mapped Location
0,17-1156755,05/04/2017,05814005400,121 QUEENS LN,NASHVILLE,TN,37218.0,"LUSTER, PATRICIA",EMAIL,HOW TAKEN: EMAIL\nHigh grass,DONE,1.0,07/13/2017,Violation Inspection - PS,VIORES,HIGH WEEDS,"121 QUEENS LN\nNASHVILLE, TN 37218\n(36.213718..."
1,17-1165789,08/24/2017,08116056500,1007 11TH AVE N,NASHVILLE,TN,37208.0,"SARGENT, THOMAS JR. & CHRISTINE",OTHER SOURCE,High grass/weeds.,DONE,19.0,08/24/2017,Initial Violation Inspection - PS,PSNVF,NaN,"1007 11TH AVE N\nNASHVILLE, TN 37208\n(36.1702..."
2,17-1157131,05/09/2017,10703001600,715 ERMAC DR,NASHVILLE,TN,37214.0,"H.H. HERITAGE INN OF NASHVILLE, LLC",PHONE,High grass & weeds,DONE,15.0,07/13/2017,Env Court 1st Setting - PS,DMJCFINE,HIGH WEEDS,"715 ERMAC DR\nNASHVILLE, TN 37214\n(36.146283,..."
3,17-1161502,07/06/2017,07116009000,903 BLUE RIDGE DR,NASHVILLE,TN,37207.0,"SIMMONS, MABLINE P.",PHONE,Inoperable vehicle on row.,DONE,5.0,08/24/2017,Initial Violation Inspection - PS,PSNVF,NaN,"903 BLUE RIDGE DR\nNASHVILLE, TN 37207\n(36.19..."
4,17-1153921,03/06/2017,11813018100,3605B MAYFLOWER PL,NASHVILLE,TN,37204.0,"ROBISON, ALEXANDER B. ET UX",PHONE,"Junk, trash & debris",DONE,25.0,04/06/2017,Violation Inspection - PS,VIORES,"OPEN STORAGE, JUNK, TRASH, DEBRIS","3605B MAYFLOWER PL\nNASHVILLE, TN 37204\n(36.1..."
5,17-1156955,05/08/2017,12001009000,1202 VULTEE BLVD,NASHVILLE,TN,37217.0,"BARRETT, MALCOLM L. JR.",PHONE,Couch in the front ditch,DONE,13.0,07/13/2017,Env Court 1st Setting - PS,DMJUSTCT,"OPEN STORAGE, JUNK, TRASH, DEBRIS","1202 VULTEE BLVD\nNASHVILLE, TN 37217\n(36.123..."
6,17-1165799,08/24/2017,10409011200,3723 RICHLAND AVE,NASHVILLE,TN,37205.0,"DUNDON, THOMAS H. & LEAH A.",Codes Online,advertising more guest than allowed on permit,OPEN,24.0,NaN,NaN,NaN,NaN,"3723 RICHLAND AVE\nNASHVILLE, TN 37205\n(36.13..."
7,17-1154514,03/20/2017,11613007600,5971 POST RD,NASHVILLE,TN,37205.0,"ELLIS, STEPHEN NICHOLAS & ODOM, KIMBERLY SUZANNE",PHONE,mattresses and trash on side of road.,DONE,23.0,04/06/2017,Violation Inspection - PS,VIORES,"OPEN STORAGE, JUNK, TRASH, DEBRIS","5971 POST RD\nNASHVILLE, TN 37205\n(36.107168,..."
8,15-1114246,02/11/2015,08102009600,1713 NUBELL ST,NASHVILLE,TN,37208.0,FREEDOM PROPERTIES,INSPECTOR REPORTED,VACANT/UNSECURED,DONE,2.0,12/08/2016,Final Sign-off - PS,VIORES,"EXTERIOR REPAIR,INTERIOR REPAIR","1713 NUBELL ST\nNASHVILLE, TN 37208\n(36.18863..."
9,17-1153010,02/10/2017,07103005200,2402 DICKERSON PIKE,NASHVILLE,TN,37207.0,"ALBARATI, AZIZ M. & HAUTER, AMIN",PHONE,Garage door unsecure - vacant building; food t...,DONE,5.0,07/13/2017,Env Court 1st Setting - PS,DMJCFINE,"Exterior of Buildings � Doors,USE NOT PERMITTED","2402 DICKERSON PIKE\nNASHVILLE, TN 37207\n(36...."


In [3]:
Standards

,Request #,Date Received,Property APN,Property Address,City,State,ZIP,Property Owner,Complaint Source,Reported Problem,Status,Council District,Last Activity Date,Last Activity,Last Activity Result,Violations Noted,Mapped Location
0,17-1156755,05/04/2017,05814005400,121 QUEENS LN,NASHVILLE,TN,37218.0,"LUSTER, PATRICIA",EMAIL,HOW TAKEN: EMAIL\nHigh grass,DONE,1.0,07/13/2017,Violation Inspection - PS,VIORES,HIGH WEEDS,"121 QUEENS LN\nNASHVILLE, TN 37218\n(36.213718..."
1,17-1165789,08/24/2017,08116056500,1007 11TH AVE N,NASHVILLE,TN,37208.0,"SARGENT, THOMAS JR. & CHRISTINE",OTHER SOURCE,High grass/weeds.,DONE,19.0,08/24/2017,Initial Violation Inspection - PS,PSNVF,NaN,"1007 11TH AVE N\nNASHVILLE, TN 37208\n(36.1702..."
2,17-1157131,05/09/2017,10703001600,715 ERMAC DR,NASHVILLE,TN,37214.0,"H.H. HERITAGE INN OF NASHVILLE, LLC",PHONE,High grass & weeds,DONE,15.0,07/13/2017,Env Court 1st Setting - PS,DMJCFINE,HIGH WEEDS,"715 ERMAC DR\nNASHVILLE, TN 37214\n(36.146283,..."
3,17-1161502,07/06/2017,07116009000,903 BLUE RIDGE DR,NASHVILLE,TN,37207.0,"SIMMONS, MABLINE P.",PHONE,Inoperable vehicle on row.,DONE,5.0,08/24/2017,Initial Violation Inspection - PS,PSNVF,NaN,"903 BLUE RIDGE DR\nNASHVILLE, TN 37207\n(36.19..."
4,17-1153921,03/06/2017,11813018100,3605B MAYFLOWER PL,NASHVILLE,TN,37204.0,"ROBISON, ALEXANDER B. ET UX",PHONE,"Junk, trash & debris",DONE,25.0,04/06/2017,Violation Inspection - PS,VIORES,"OPEN STORAGE, JUNK, TRASH, DEBRIS","3605B MAYFLOWER PL\nNASHVILLE, TN 37204\n(36.1..."
5,17-1156955,05/08/2017,12001009000,1202 VULTEE BLVD,NASHVILLE,TN,37217.0,"BARRETT, MALCOLM L. JR.",PHONE,Couch in the front ditch,DONE,13.0,07/13/2017,Env Court 1st Setting - PS,DMJUSTCT,"OPEN STORAGE, JUNK, TRASH, DEBRIS","1202 VULTEE BLVD\nNASHVILLE, TN 37217\n(36.123..."
6,17-1165799,08/24/2017,10409011200,3723 RICHLAND AVE,NASHVILLE,TN,37205.0,"DUNDON, THOMAS H. & LEAH A.",Codes Online,advertising more guest than allowed on permit,OPEN,24.0,NaN,NaN,NaN,NaN,"3723 RICHLAND AVE\nNASHVILLE, TN 37205\n(36.13..."
7,17-1154514,03/20/2017,11613007600,5971 POST RD,NASHVILLE,TN,37205.0,"ELLIS, STEPHEN NICHOLAS & ODOM, KIMBERLY SUZANNE",PHONE,mattresses and trash on side of road.,DONE,23.0,04/06/2017,Violation Inspection - PS,VIORES,"OPEN STORAGE, JUNK, TRASH, DEBRIS","5971 POST RD\nNASHVILLE, TN 37205\n(36.107168,..."
8,15-1114246,02/11/2015,08102009600,1713 NUBELL ST,NASHVILLE,TN,37208.0,FREEDOM PROPERTIES,INSPECTOR REPORTED,VACANT/UNSECURED,DONE,2.0,12/08/2016,Final Sign-off - PS,VIORES,"EXTERIOR REPAIR,INTERIOR REPAIR","1713 NUBELL ST\nNASHVILLE, TN 37208\n(36.18863..."
9,17-1153010,02/10/2017,07103005200,2402 DICKERSON PIKE,NASHVILLE,TN,37207.0,"ALBARATI, AZIZ M. & HAUTER, AMIN",PHONE,Garage door unsecure - vacant building; food t...,DONE,5.0,07/13/2017,Env Court 1st Setting - PS,DMJCFINE,"Exterior of Buildings � Doors,USE NOT PERMITTED","2402 DICKERSON PIKE\nNASHVILLE, TN 37207\n(36...."
